In [2]:
import EXOSIMS as EX, os.path
import EXOSIMS.MissionSim as msim
import EXOSIMS.StarCatalog.EXOCAT1 as exc
import numpy as np
import math as ma
import matplotlib.pyplot as plt
import astropy.units as u
import pickle, json, warnings, astropy
%matplotlib

Using matplotlib backend: TkAgg


###  Use script files saved in EXOSIMS path

In [3]:
# LOAD JSON SCRIPT FILE
#jfile = 'template_WFIRST_KeplerLike.json'
#jfile = 'template_WFIRST_EarthTwinHabZone.json'
#jfile = 'template_WFIRST_KnownRV.json'
jfile = 'template_rpateltest_KnownRV.json'
scriptfile = os.path.join(EX.__path__[0],'Scripts',jfile)

In [4]:
script = open(scriptfile).read()
specs_from_file = json.loads(script)

In [5]:
# QUESTION -- DO I HAVE TO RUN THIS EACH TIME OR IS THERE A WAY TO SAVE/LOAD THE OUTPUT?
%time sim = msim.MissionSim(scriptfile)

MissionSim.pyc: Beginning logging to "rpatel_log.txt" at level INFO
MissionSim: Seed is:  286188818
Imported SurveyEnsemble (prototype module) from EXOSIMS.Prototypes.SurveyEnsemble
Imported KnownRVPlanetsUniverse (specific module) from EXOSIMS.Simulat[...]se.KnownRVPlanetsUniverse
Imported KnownRVPlanetsTargetList (specific module) from EXOSIMS.TargetList.KnownRVPlanetsTargetList
Imported StarCatalog (prototype module) from EXOSIMS.Prototypes.StarCatalog
Imported Stark (specific module) from EXOSIMS.ZodiacalLight.Stark
Imported WFIRSTOpticalSystem (specific module) from EXOSIMS.OpticalSystem.WFIRSTOpticalSystem
Imported PostProcessing (prototype module) from EXOSIMS.Prototypes.PostProcessing
Imported GalaxiesFaintStars (specific module) from EXOSIMS.BackgroundSources.GalaxiesFaintStars
Imported BrownCompleteness (specific module) from EXOSIMS.Completeness.BrownCompleteness
Imported KnownRVPlanets (specific module) from EXOSIMS.PlanetPopulation.KnownRVPlanets
Imported FortneyMarleyCaho

Imported TimeKeeping (prototype module) from EXOSIMS.Prototypes.TimeKeeping
Imported SurveySimulation (prototype module) from EXOSIMS.Prototypes.SurveySimulation
CPU times: user 1h 25min 8s, sys: 10min 5s, total: 1h 35min 13s
Wall time: 2h 2min 22s


In [12]:
%time sim.SurveySimulation.run_sim()

Current mission time:  0.0 d


/Users/rpatel/Documents/envs/py27/lib/python2.7/site-packages/astropy/units/quantity.py:841: RuntimeWarning: divide by zero encountered in true_divide
  return super(Quantity, self).__truediv__(other)


Current mission time:  1.12493453415 d
Current mission time:  2.16626982299 d
Current mission time:  5.46291443034 d
Current mission time:  6.4630578571 d
Current mission time:  7.69879943601 d
Current mission time:  8.70015039416 d
Current mission time:  9.70502778317 d
Current mission time:  10.7063909151 d
Current mission time:  11.7308165606 d
Current mission time:  12.730926201 d
Current mission time:  84.6269356979 d
Current mission time:  91.9124136504 d
Current mission time:  92.9125828152 d
Current mission time:  93.9141891297 d
Current mission time:  95.1255920192 d
Current mission time:  96.1258024269 d
Current mission time:  97.1262776025 d
Current mission time:  169.00023075 d
Current mission time:  174.446185604 d
Current mission time:  175.47840714 d
Current mission time:  176.481249945 d
Current mission time:  177.482867487 d
Current mission time:  261.075208856 d
Current mission time:  262.268020387 d
Current mission time:  263.269191634 d
Current mission time:  339.48

'Simulation results in .DRM'

## Module Objects

In [244]:
# Stored detection information -- if any -- in DRM Dictionary 
DRM = sim.SurveySimulation.DRM

In [224]:
# Simulation specifications ; i.e., all parameters used in simulation
AllSpecs = sim.genOutSpec()

The slowest run took 13.10 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 22.5 µs per loop


# Right now, I'm loading obvious objects into local variables
# We can add more depending on what we need for the interface

### modules:

In [237]:
TL = sim.TargetList
SC = sim.StarCatalog
SU = sim.SimulatedUniverse
SSim = sim.SurveySimulation
OS = sim.OpticalSystem
ZL = sim.ZodiacalLight
BS = sim.BackgroundSources
CP = sim.Completeness
PP = sim.PlanetPopulation
PM = sim.PlanetPhysicalModel

## Stellar Properties

In [213]:
Name   = TL.Name
Spec   = TL.Spec
parx   = TL.parx
Umag   = TL.Umag
Bmag   = TL.Bmag
Vmag   = TL.Vmag
Rmag   = TL.Rmag
Imag   = TL.Imag
Jmag   = TL.Jmag
Hmag   = TL.Hmag
Kmag   = TL.Kmag
dist   = TL.dist
BV = TL.BV
MV = TL.MV
BC = TL.BC
L  = TL.L
coords = TL.coords
pmra   = TL.pmra
pmdec  = TL.pmdec
rv = TL.rv
Binary_Cut = TL.Binary_Cut
maxintTime = TL.maxintTime
comp0  = TL.comp0
MsEst  = TL.MsEst
MsTrue = TL.MsTrue
nStars = TL.nStars

star_prop = {'Name':Name,'Spec':Spec,'parx':parx,'Umag':Umag,'Bmag':Bmag,'Vmag':Vmag,
            'Imag':Imag,'Jmag':Jmag,'Hmag':Hmag,'Kmag':Kmag,'dist':dist,'BV':BV,
            'MV':MV,'Lum':L,'coords':coords,'pmra':pmra,'pmdec':pmdec,'rv':rv,
            'Binary_Cut':Binary_Cut,'maxintTime':maxintTime,'comp0':comp0,'MsEst':MsEst,
            'MsTrue':MsTrue,'nStars':nStars}


## Planets Generated Properties

In [217]:
try:
    arange = TL.arange
    erange = TL.erange
    wrange = TL.wrange
    Orange = TL.Orange
    prange = TL.prange
    Irange = TL.Irange
    Rrange, Mprange = TL.Rrange, TL.Mprange
    rrange = TL.rrange
    synplanet_prop = {'arange':arange,'erange':erange,'wrange':wrange,'Orange':Orange,
                      'prange':prange,'Irange':Irange,'Rrange':Rrange,'Mprange':Mprange,
                      'rrange':rrange}
except AttributeError:
    print 'Sorry.. no go.'
    print 'Not simulated planets.'
    synplanet_prop = None

try:
    nPlans, plan2star = SU.nPlans, SU.plan2star
    sInds = SU.sInds
    # ORBTIAL PARAMETERS
    sma,e,w,O,I = SU.a, SU.e, SU.w, SU.O, SU.I
    # PLANET PROPERTIES
    Mp,Rp = SU.Mp, SU.Rp
    # POSITION AND VELOCITY VECTOR OF PLANET
    r, v = SU.r, SU.v
    # ALBEDO
    p = SU.p
    fEZ = SU.fEZ

    empplanet_prop = {'nplans':nPlans,'plan2star':plan2star,'sInds':sInds,'sma':sma,
                      'e':e,'w':w,'O':O,'I':I,'Mp':Mp,'Rp':Rp,'r':r,'v':v,'p':p,'fEZ':fEZ}
except AttributeError:
    print 'Sorry.. no go.'
    print 'Not ``real`` planets.'
    empplanet_prop = None
    

Sorry.. no go.
Not simulated planets.


## Additional simulation results -- add more as needed

In [253]:
# number of visits to each star -- double check
nvisits_perstar = SU.Completeness.visits

etc_data = {'nvisits':nvisits_perstar}

## Dump Survey Simulation Results -- not completeness

In [258]:
mlife = AllSpecs['missionLife']
nsimplanets = specs_from_file['Nplanets']
jfilebase = jfile.strip('.json').strip('template_')

simresults = 'simresults_%iyrs_%.0Estars_%s.pickle' %(mlife,nsimplanets,jfilebase)
simfile = os.path.join('/Users/rpatel/Dropbox/Research/WFIRST/EXOSIMSTesting/SimResults/',simresults)

data = {'DRM':DRM,'empplanet_prop':empplanet_prop,'synplanet_prop':synplanet_prop,
        'star_prop':star_prop,'AllSpecs':AllSpecs,'etc_data':etc_data}

handler = open(simfile,'wb')
pickle.dump(data,handler)
handler.close()